<a href="https://colab.research.google.com/github/laffertybrian/Math-Skill-Builder/blob/main/Little_Professor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile professor.py
from pandas.io.common import file_exists
import random, csv, os
import pandas as pd

# variables for file storage
game_results_storage = "little_professor_game_resutls.csv"
player_settings_storage = "player_settings.csv"
FOLDER = "/content/drive/MyDrive/Data Science Projects/Math_Skill_Builder/Data/"

# creating folder for storage
os.makedirs(FOLDER, exist_ok = True)

game_results_fieldnames = ['Player_Name', 'First_Number', 'Second_Number', 'Answer', 'Operator', 'Player_Answer', 'Correct', 'Prediction']
player_settings_fieldnames = ['Player_Name', 'Score', 'Level', 'Rounds', 'Operator']

def main():

    welcome()

    final_score = 0

    for _ in range(player.rounds):
        details = create_problem(player.level, player.operator)
        score = check_solution(*details)
        final_score += score
        

    print(f'Score: {final_score}')

def welcome():
  global player
  print('Welcome to Little Professor! A math skill building game.')
  player_name = input('Enter your player name, or if you are new here type New: ')

  if player_name.lower() == 'new':
    player = Player.get()
    record_to_file(game_results_storage, player.export(), player_settings_fieldnames)
    return

  if file_check(game_results_storage):
    df = retrieve_from_file(game_results_storage)
    if player_name in df['Player_Name'].tolist():
      print("Welcome Back")
      used_last = input('Would you like to use the settings from your last game? (Y or N)')
      if used_last.lower() == 'y':
        used_last_df = retrieve_from_file(game_results_storage)
        player_set = used_last_df[used_last_df['Player_Name'] == player_name].sort_index(ascending = False)   
        sorted_dict = player_set.to_dict('records')
        converted = number_convert(sorted_dict[0])
        player = Player(**converted)
        return
      if used_last.lower() == 'n':
        player = Player(Player_Name = player_name)
        player.reset_settings()
        update_dataframe(player.export())
        return
      else:
        no_name()

    else:
      no_name()

  else:
    no_name()

def no_name():
  print("We did not recognize your name. Let me ask you some questions and get you started!")
  player = Player.get()
  record_to_file(game_results_storage, player.export(), player_settings_fieldnames)

def get_name():
  while True:
    name = input('Name: ')

    if file_check(game_results_storage): 
      df = retrieve_from_file(game_results_storage)

      if name in df['Player_Name'].tolist():
        print('Another player has that name, choose another.')
      else:
        return name
    else:
      return name


def get_level():
    print('Type 1 for single, 2 for double, or 3 for triple digit numbers!')
    print('Or 4 to customize your number range.')
    start_settings = []
    while True:
        try:
            level = int(input('Level: '))
            if level in [1, 2, 3, 4]:
              break
        except ValueError:
            pass
    start_settings.append(level)            
    if start_settings[0] == 4:
        while True:
          try:
            num_range = input('Number range X-Y: ').split('-')
            cus_x = int(num_range[0])
            start_settings.append(cus_x)
            cus_y = int(num_range[1])
            start_settings.append(cus_y)
            break
          except ValueError:
            pass
    return start_settings

def get_rounds():
  while True:
    try:
      rounds = int(input('How many questions would you like? '))
      return rounds
    except ValueError:
      pass

def get_operator():
  while True:
    try:
      operator = input("Pick your operator [+, -, *, /]: ")
      if operator in ['+', '-', '*', '/']:
        return operator
    except ValueError:
      pass


def generate_integer(level):
    if level[0] == 1:
        return random.randint(0,9)
    if level[0] == 2:
        return random.randint(10, 99)
    if level[0] == 3:
        return random.randint(100, 999)
    if level[0] == 4:
        return random.randint(level[1], level[2])

def create_problem(level, operator):
    x = generate_integer(level)
    y = generate_integer(level)

    if operator != '/':
      answer = int(input(f'{x} {operator} {y} = '))

      if operator == '+':
        ans = x + y
      elif operator == '-':
        ans = x - y
      elif operator == '*':
        ans = x * y

      return [x, y, ans, answer, operator]
      
    else:
      answer = int(input(f'{x * y} {operator} {y} = '))

      product = x * y
      ans = x

      return [product, y, ans, answer, operator]

def check_solution(first, second, solution, response, operator):
    tries = 0

    if solution == response:
        return 1
    else:
        while tries < 2:
            print('EEE')
            again = int(input(f'{first} {operator} {second} = '))
            if again == solution:
                return 0
            tries += 1

    print('EEE')
    print(f'Correct answer: {first} {operator} {second} = {solution}')
    return 0

# function to convert strings to integers after retrieval from dataframe
def number_convert(dict_):
  converted = []
  convert = dict_

  num_list = dict_['Level']
  if dict_['Level'].startswith('['):
    num_list = num_list.strip('[')
    num_list = num_list.strip(']')
    num_list = num_list.split(',')

    for _ in num_list:
      _.replace(' ', '')
      converted.append(int(_))
  convert['Level'] = converted

  round = dict_['Rounds']
  convert['Rounds'] = int(round)

  return convert

# creating a function to check if a file exsits
def file_check(file):
  folderfile = FOLDER + file
  file_exists = os.path.isfile(folderfile)
  if file_exists:
    return True
  else:
    return False

def record_to_file(file, results, fieldnames, edit = 'a'):
  folderfile = FOLDER + file
  if file_check(file):
    with open(folderfile, edit, newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      if edit == 'w':
        writer.writeheader()
        for row in results:
          writer.writerow(row)
      else:
        writer.writerow(results)

  else:
    with open(folderfile, 'w', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writeheader()
      writer.writerow(results)

def retrieve_from_file(file):
  folderfile = FOLDER + file
  with open(folderfile, 'r') as f:
    reader = csv.DictReader(f)
    results_df = pd.DataFrame(reader)
  return results_df


def update_dataframe(info):
  df = retrieve_from_file(game_results_storage)
  players_index = df.index[df['Player_Name'] == info['Player_Name']]
  index_num = list(players_index).pop(0)
  for key, value in info.items():
    df.at[index_num, key] = value

  df_dictionaries = df.to_dict('records')
  record_to_file(game_results_storage, df_dictionaries, player_settings_fieldnames, 'w')


class Player:

  def __init__(self, Player_Name, Level = 0, Rounds = 0, Operator = '', Score = 0):
    self.name = Player_Name
    self.level = Level
    self.rounds = Rounds
    self.operator = Operator

  def __str__(self):
    ...
  
  @classmethod
  def get(cls):
    name = get_name()
    level = get_level()
    rounds = get_rounds()
    operator = get_operator()
    return cls(name, level, rounds, operator)

  def reset_settings(self):
    self.level = get_level()
    self.rounds = get_rounds()
    self.operator = get_operator()

  def export(self):
    return {'Player_Name': self.name, 'Level': self.level, 'Rounds': self.rounds, 'Operator': self.operator}

if __name__ == "__main__":
    main()

Writing professor.py


In [2]:
%run professor.py

Welcome to Little Professor! A math skill building game.
Enter your player name, or if you are new here type New: Brian
Welcome Back
Would you like to use the settings from your last game? (Y or N)N
Type 1 for single, 2 for double, or 3 for triple digit numbers!
Or 4 to customize your number range.
Level: 4
Number range X-Y: 1-9
How many questions would you like? 4
Pick your operator [+, -, *, /]: *
6 * 7 = 42
4 * 4 = 16
3 * 4 = 12
5 * 2 = 10
Score: 4
